# Validate calibration of phones

This notebook retrieves the calibration results for a particular experiment and validates them

## Experiment parameters

If you only want to run experiments, you only need to edit these variables. The notebook will retrieve the appropriate spec, use it to find the calibration periods from the database and validate the calibration. You probably want to publish this notebook along with your results so that others can examine it as well.

In [ ]:
DATASTORE_URL = "http://cardshark.cs.berkeley.edu"
AUTHOR_EMAIL = "shankari@eecs.berkeley.edu" # e.g. shankari@eecs.berkeley.edu
CURR_SPEC_ID = "sfba_moving_calibration_only_1" # e.g. sfba_calibration_only_1
MAX_DURATION_VARIATION = 5 * 60 # seconds

## Setup some basic imports

In [ ]:
import logging
logging.getLogger().setLevel(logging.DEBUG)

In [ ]:
import copy
import matplotlib.pyplot as plt
import arrow
import pandas as pd
%matplotlib notebook

In [ ]:
import requests

In [ ]:
import folium
import folium.features as fof
import folium.plugins as fpl
import folium.utilities as ful
import branca.element as bre

## Define some simple utility functions

In [ ]:
def get_row_count(n_maps, cols):
    rows = int(n_maps / cols)
    if (n_maps % cols != 0):
        rows = rows + 1
    return rows

## Setup the ability to make calls to the server

In [ ]:
def retrieve_data_from_server(user_label, key_list, start_ts, end_ts):
    post_msg = {
        "user": user_label,
        "key_list": key_list,
        "start_time": start_ts,
        "end_time": end_ts
    }
    # print("About to retrieve messages using %s" % post_msg)
    response = requests.post(DATASTORE_URL+"/datastreams/find_entries/timestamp", json=post_msg)
    # print("response = %s" % response)
    response.raise_for_status()
    ret_list = response.json()["phone_data"]
    # print("Found %d entries" % len(ret_list))
    return ret_list

def retrieve_all_data_from_server(user_label, key_list):
    return retrieve_data_from_server(user_label, key_list, 0, arrow.get().timestamp)

## Find the current spec

In [ ]:
all_spec_entry_list = retrieve_all_data_from_server(AUTHOR_EMAIL, ["config/evaluation_spec"])
curr_spec_entry = None
for s in all_spec_entry_list:
    if s["data"]["label"]["id"] == CURR_SPEC_ID:
        curr_spec_entry = s
curr_spec_wrapper = curr_spec_entry["data"]
curr_spec = curr_spec_wrapper["label"]
curr_spec["name"]

## Find all evaluation transitions within the start and end times of this spec

In [ ]:
eval_start_ts = curr_spec_wrapper["start_ts"]
eval_end_ts = curr_spec_wrapper["end_ts"]
eval_tz = curr_spec["region"]["timezone"]
print("Evaluation ran from %s -> %s" % (arrow.get(eval_start_ts).to(eval_tz), arrow.get(eval_end_ts).to(eval_tz)))
phone_labels = curr_spec["phones"]

Data model here is:

```
eval_transitions
    - android
        - ucb.sdb.android.1
            - list of evaluation transitions
        - ....
    - ios
```

In [ ]:
eval_transitions = copy.deepcopy(phone_labels)
for phoneOS, phone_map in eval_transitions.items():
    print("Reading data for %s phones" % phoneOS)
    for phone_label in phone_map:
        print("Loading transitions for phone %s" % phone_label)
        curr_phone_transitions = retrieve_data_from_server(phone_label, ["manual/evaluation_transition"], eval_start_ts, eval_end_ts)
        curr_phone_role = phone_map[phone_label]
        phone_map[phone_label] = {"role": curr_phone_role}
        phone_map[phone_label]["transitions"] = curr_phone_transitions

### Find calibration transitions, validate and map them to ranges

From here onwards, we will add the results of manipulation to each phone entry - e.g.

```
eval_transitions
    - android
        - ucb.sdb.android.1
            - transitions (all transition entries, added in previous step)
            - calibration_transitions (calibration transitions, will be added in this step)
        - ....
    - ios
```

In [ ]:
for phoneOS, phone_map in eval_transitions.items():
    print("Processing data for %s phones" % phoneOS)
    for phone_label in phone_map:
        print("Processing transitions for phone %s" % phone_label)
        curr_phone_transitions = [t["data"] for t in phone_map[phone_label]["transitions"]]
        # print(curr_phone_transitions)
        curr_calibration_transitions = [t for t in curr_phone_transitions if (t["transition"] in ["START_CALIBRATION_PERIOD", "STOP_CALIBRATION_PERIOD", 0, 1]) and ("AO" in t["trip_id"])]
        print("Filtered %d total -> %d calibration transitions " % (len(curr_phone_transitions), len(curr_calibration_transitions)))
        phone_map[phone_label]["calibration_transitions"] = sorted(curr_calibration_transitions, key=lambda t:t["ts"])

In [ ]:
first_two_entries_list = []
for phoneOS, phone_map in eval_transitions.items():
    print("Processing data for %s phones" % phoneOS)
    for phone_label in phone_map:
        print("Processing transitions for phone %s" % phone_label)
        first_two_entries_list.extend(sorted(phone_map[phone_label]["transitions"], key=lambda t:t["data"]["ts"])[1:2])

In [ ]:
print("\n".join([str((t["_id"], t["data"]["transition"], t["data"]["trip_id"], arrow.get(t["data"]["ts"]).to(eval_tz).format())) for t in first_two_entries_list]))

In [ ]:
test_df = pd.DataFrame({})
test_df["hr"] = []
test_df

In [ ]:
print("\n".join([str((t["_id"], t["data"]["transition"], t["data"]["trip_id"], arrow.get(t["data"]["ts"]).to(eval_tz).format())) for t in first_two_entries_list]))

In [ ]:
len(first_two_entries_list)

In [ ]:
print(",".join([str('boi.ObjectId("%s")' % t["_id"]["$oid"]) for t in first_two_entries_list]))

In [ ]:
ios_1_transitions = eval_transitions["ios"]["ucb-sdb-ios-2"]["calibration_transitions"]
print("\n".join([str((t["transition"], t["trip_id"], t["ts"], arrow.get(t["ts"]).to(eval_tz))) for t in ios_1_transitions]))

In [ ]:
arrow.get("2019-07-12T10:17:22.232000-07:00").timestamp

In [ ]:
ios_1_transitions = eval_transitions["android"]["ucb-sdb-android-1"]["calibration_transitions"]
print("\n".join([str((t["transition"], t["trip_id"], t["ts"], arrow.get(t["ts"]).to(eval_tz))) for t in ios_1_transitions]))

In [ ]:
# We expect that transitions occur in pairs
def transitions_to_ranges(transition_list, start_tt, end_tt, start_ti, end_ti):
    start_transitions = transition_list[::2]
    end_transitions = transition_list[1::2]
    print("\n".join([str((t["transition"], t["trip_id"], t["ts"], arrow.get(t["ts"]).to(eval_tz))) for t in start_transitions]))
    print("\n".join([str((t["transition"], t["trip_id"], t["ts"], arrow.get(t["ts"]).to(eval_tz))) for t in end_transitions]))
    
    if len(transition_list) % 2 == 0:
        print("All calibration is complete, nothing to change")
    else:
        print("Ongoing calibration, adding fake end transition")
        last_start_transition = phone_map[phone_label]["transitions"][-1]
        fake_end_transition = copy.copy(last_start_transition)
        fake_end_transition["data"]["transition"] = end_tt
        curr_ts = arrow.get().timestamp
        fake_end_transition["data"]["ts"] = curr_ts
        if "fmt_time" in last_start_transition["data"]:
            fake_end_transition["data"]["fmt_time"] = arrow.get(curr_ts).to(eval_tz)
        fake_end_transition["metadata"]["write_ts"] = curr_ts
        if "write_fmt_time" in last_start_transition["metadata"]:
            fake_end_transition["metadata"]["write_fmt_time"] = arrow.get(curr_ts).to(eval_tz)
        fake_end_transition["metadata"]["platform"] = "fake"
        if "local_dt" in fake_end_transition["data"]:
            del fake_end_transition["data"]["local_dt"]
        transition_list.append(fake_end_transition)
        end_transitions.append(fake_end_transition["data"])
            
    print("\n".join([str((t["transition"], t["trip_id"], t["ts"], arrow.get(t["ts"]).to(eval_tz))) for t in start_transitions]))
    print("\n".join([str((t["transition"], t["trip_id"], t["ts"], arrow.get(t["ts"]).to(eval_tz))) for t in end_transitions]))

    range_list = []
    for (s, e) in zip(start_transitions, end_transitions):
        # print("------------------------------------- \n %s -> \n %s" % (s, e))
        assert s["transition"] == start_tt or s["transition"] == start_ti, "Start transition has %s transition" % s["transition"]
        assert e["transition"] == end_tt or s["transition"] == end_ti, "Stop transition has %s transition" % s["transition"]
        assert s["trip_id"] == e["trip_id"], "trip_id mismatch! %s != %s" % (s["trip_id"], e["trip_id"])
        assert e["ts"] > s["ts"], "end %s is before start %s" % (arrow.get(e["ts"]), arrow.get(s["ts"]))
        for f in ["spec_id", "device_manufacturer", "device_model", "device_version"]:
            assert s[f] == e[f], "Field %s mismatch! %s != %s" % (f, s[f], e[f])
        curr_range = {"trip_id": s["trip_id"], "start_ts": s["ts"], "end_ts": e["ts"], "duration": (e["ts"] - s["ts"])}
        range_list.append(curr_range)
        
    return range_list

In [ ]:
transitions_to_ranges(eval_transitions["ios"]["ucb-sdb-ios-1"]["calibration_transitions"], "START_CALIBRATION_PERIOD", "STOP_CALIBRATION_PERIOD", 0, 1)
# transitions_to_ranges(eval_transitions["ios"]["ucb-sdb-ios-1"]["calibration_transitions"], "START_CALIBRATION_PERIOD", "STOP_CALIBRATION_PERIOD", 0, 1, phone_map[phone_label]["transitions"][-1], eval_end_ts)

In [ ]:
for phoneOS, phone_map in eval_transitions.items():
    print("Processing data for %s phones" % phoneOS)
    for phone_label in phone_map:
        curr_calibration_ranges = transitions_to_ranges(phone_map[phone_label]["calibration_transitions"], "START_CALIBRATION_PERIOD", "STOP_CALIBRATION_PERIOD", 0, 1)
        print("Found %d ranges for phone %s" % (len(curr_calibration_ranges), phone_label))
        phone_map[phone_label]["calibration_ranges"] = curr_calibration_ranges

## Validate the ranges for individual phones

This involves two main checks:
- that we have at least one calibration range for each test in the spec. Note that we do not currently enforce that we have exactly one calibration range for each test, on the theory that more calibration is always good. But I am open to argument about this
- that the settings in the calibration range are consistent with the spec

In [ ]:
expected_config_map = {}
for ct in curr_spec["calibration_tests"]:
    expected_config_map[ct["id"]] = ct["config"]["sensing_config"]

In [ ]:
# Current accuracy constants
# Since we can't read these from the phone, we hardcoded them from the documentation
# If there are validation failures, these need to be updated
# In the future, we could upload the options from the phone (maybe the accuracy control)
# but that seems like overkill here

accuracy_options = {
    "android": {
        "PRIORITY_HIGH_ACCURACY": 100,
        "PRIORITY_BALANCED_POWER_ACCURACY": 102,
        "PRIORITY_LOW_POWER": 104,
        "PRIORITY_NO_POWER": 105
    },
    "ios": {
        "kCLLocationAccuracyBestForNavigation": -2,
        "kCLLocationAccuracyBest": -1,
        "kCLLocationAccuracyNearestTenMeters": 10,
        "kCLLocationAccuracyHundredMeters": 100,
        "kCLLocationAccuracyKilometer": 1000,
        "kCLLocationAccuracyThreeKilometers": 3000,
    }
}

In [ ]:
opt_array_idx = lambda phoneOS: 0 if phoneOS == "android" else 1

def validate_filter(phoneOS, config_during_test, expected_config):
    # filter checking is a bit tricky because the expected value has two possible values and the real config has two possible values
    expected_filter = expected_config["filter"]
    if type(expected_filter) == int:
        ev = expected_filter
    else:
        assert type(expected_filter) == list, "platform specific filters should be specified in array, not %s" % expected_filter
        ev = expected_filter[opt_array_idx(phoneOS)]
        
    if phoneOS == "android":
        cvf = "filter_time"
    elif phoneOS == "ios":
        cvf = "filter_distance"
        
    assert config_during_test[cvf] == ev, "Field filter mismatch! %s != %s" % (config_during_test, expected_config)
    
def validate_accuracy(phoneOS, config_during_test, expected_config):
    # expected config accuracy is an array of strings ["PRIORITY_BALANCED_POWER_ACCURACY", "kCLLocationAccuracyNearestTenMeters"]
    # so we find the string at the correct index and then map it to the value from the options
    ev = accuracy_options[phoneOS][expected_config["accuracy"][opt_array_idx(phoneOS)]]
    assert config_during_test["accuracy"] == ev, "Field accuracy mismatch! %s != %s" % (config_during_test[accuracy], ev)

for phoneOS, phone_map in eval_transitions.items():
    print("Processing data for %s phones" % phoneOS)
    for phone_label in phone_map:
        curr_calibration_ranges = phone_map[phone_label]["calibration_ranges"]
        all_test_ids = [r["trip_id"] for r in curr_calibration_ranges]
        unique_test_ids = sorted(list(set(all_test_ids)))
        spec_test_ids = sorted([ct["id"] for ct in curr_spec["calibration_tests"]])
        # assert unique_test_ids == spec_test_ids, "Missing calibration test while comparing %s, %s" % (unique_test_ids, spec_test_ids)
        for r in curr_calibration_ranges:
            config_during_test_entries = retrieve_data_from_server(phone_label, ["config/sensor_config"], r["start_ts"], r["end_ts"])
            print("%s -> %s" % (r["trip_id"], [c["data"]["accuracy"] for c in config_during_test_entries]))
            # assert len(config_during_test_entries) == 1, "Out of band configuration? Found %d config changes" % len(config_during_test_entries)
            # config_during_test = config_during_test_entries[0]["data"]
            # sexpected_config = expected_config_map[r["trip_id"]]
            # print(config_during_test, expected_config)
            # validate_filter(phoneOS, config_during_test, expected_config)
            # validate_accuracy(phoneOS, config_during_test, expected_config)
            # for f in expected_config:
            #     if f != "accuracy" and f != "filter":
            #         assert config_during_test[f] == expected_config[f], "Field %s mismatch! %s != %s" % (f, config_during_test[f], expected_config[f])

## Validate ranges across phones

This effectively has one test right now - is the duration of the tests across phones consistent?
TODO: We should add a reasonable fuzz factor based on real calibration.

We are going to create a pandas dataframe with the following structure

```
                    android_<phone_1> android_<phone_2> android_<phone_3> ....
<trip_id_1>
<trip_id_2>
...
```

Then, we can transpose it to get

```
                    <trip_id_1> <trip_id_2> <trip_id_3> ....
android_<phone_1>
android_<phone_2>
...
```

then, we can get a series of durations for each `trip_id` as a series and compare it

In [ ]:
duration_map = {}
for phoneOS, phone_map in eval_transitions.items():
    print("Processing data for %s phones" % phoneOS)
    for phone_label in phone_map:
        curr_phone_duration_map = {}
        curr_calibration_ranges = phone_map[phone_label]["calibration_ranges"]
        for r in curr_calibration_ranges:
            curr_phone_duration_map[r["trip_id"]] = r["duration"]
        duration_map[phoneOS+"_"+phone_label] = curr_phone_duration_map
        
duration_df = pd.DataFrame(duration_map).transpose()
duration_df

Since these are not statistical samples, the regular standard deviation/variation don't have much meaning. The variation is really caused by human control of the evaluation start/stop and the durations should be within a few minutes of each other. The expected variation defined in `MAX_DURATION_VARIATION`

In [ ]:
for col in duration_df:
    duration_variation = duration_df[col] - duration_df[col].median()
    print("For %s, duration_variation = %s" % (col, duration_variation.tolist()))
    assert duration_variation.abs().max() < MAX_DURATION_VARIATION,\
        "INVALID: for %s, duration_variation.abs().max() %d > threshold %d" % (col, duration_variation.abs().max(), MAX_DURATION_VARIATION)

Now, we can evaluate the actual values

## Battery drain over time

The data is in the format

```
android:
    - <phone_1>:
        - <trip_id>
          - dataframe with index = ts, columns = other fields
    - <phone_2>:
        - <trip_id>
          - dataframe with index = ts, columns = other fields
...
ios:
    - <phone_1>:
        - <trip_id>
          - dataframe with index = ts, columns = other fields
    - <phone_2>:
        - <trip_id>
          - dataframe with index = ts, columns = other fields
```

In [ ]:
for phoneOS, phone_map in eval_transitions.items():
    print("Processing data for %s phones" % phoneOS)
    for phone_label in phone_map:
        curr_calibration_ranges = phone_map[phone_label]["calibration_ranges"]
        for r in curr_calibration_ranges:
            battery_entries = retrieve_data_from_server(phone_label, ["background/battery"], r["start_ts"], r["end_ts"])
            # ios entries before running the pipeline are marked with battery_level_ratio, which is a float from 0 ->1
            # convert it to % to be consistent with android and easier to understand
            if phoneOS == "ios":
                for e in battery_entries:
                    if "battery_level_pct" not in e["data"]:
                        e["data"]["battery_level_pct"] = e["data"]["battery_level_ratio"] * 100
                        del e["data"]["battery_level_ratio"]
            battery_df = pd.DataFrame([e["data"] for e in battery_entries])
            battery_df["hr"] = (battery_df.ts-r["start_ts"])/3600.0
            r["battery_df"] = battery_df

In [ ]:
test_battery_df = eval_transitions["android"]["ucb-sdb-android-3"]["calibration_ranges"][0]["battery_df"]; test_battery_df

In [ ]:
test_battery_df = eval_transitions["android"]["ucb-sdb-android-4"]["calibration_ranges"][1]["battery_df"]; test_battery_df

In [ ]:
def plot_calibration_curves(ax, phone_map):
    for phone_label in phone_map:
        curr_calibration_ranges = phone_map[phone_label]["calibration_ranges"]
        for r in curr_calibration_ranges:
            battery_df = r["battery_df"]
            ret_axes = battery_df.plot(x="hr", y="battery_level_pct", ax=ax, label=phone_label+"_"+r["trip_id"], ylim=(0,100), sharey=True)

In [ ]:
(ifig, [android_ax, ios_ax]) = plt.subplots(ncols=1, nrows=2, figsize=(25,25))

plot_calibration_curves(ios_ax, eval_transitions["ios"])
ios_ax.legend(loc="center left", bbox_to_anchor=(1, 0.5))
plot_calibration_curves(android_ax, eval_transitions["android"])
android_ax.legend(loc="center left", bbox_to_anchor=(1, 0.5))

In [ ]:
def plot_separate_calibration_curves(fig, nRows, phone_map):
    for i, phone_label in enumerate(phone_map.keys()):
        ax = fig.add_subplot(nRows, 4, i+1, title=phone_label)
        curr_calibration_ranges = phone_map[phone_label]["calibration_ranges"]
        for r in curr_calibration_ranges:
            battery_df = r["battery_df"]
            # print(battery_df.battery_level_pct.tolist())
            battery_df.plot(x="hr", y="battery_level_pct", ax=ax, label=r["trip_id"], ylim=(0,100), sharey=True, legend=False)
    handles, labels = ax.get_legend_handles_labels()
    fig.legend(handles, labels, loc='upper center')

In [ ]:
(ifig, ax) = plt.subplots(figsize=(16,4), nrows=0, ncols=0)
nRows = get_row_count(len(eval_transitions["ios"].keys()), 4)
print(nRows)
plot_separate_calibration_curves(ifig, nRows, eval_transitions["ios"])

In [ ]:
(ifig, ax) = plt.subplots(nrows=0, ncols=0, figsize=(16,16))
nRows = get_row_count(len(eval_transitions["android"].keys()), 2)
plot_separate_calibration_curves(ifig, nRows, eval_transitions["android"])

## Location points over time

Now, we download the location points and check to see that the density is largely consistent

In [ ]:
for phoneOS, phone_map in eval_transitions.items():
    print("Processing data for %s phones" % phoneOS)
    for phone_label in phone_map:
        curr_calibration_ranges = phone_map[phone_label]["calibration_ranges"]
        for r in curr_calibration_ranges:
            all_done = False
            location_entries = []
            curr_start_ts = r["start_ts"]
            prev_retrieved_count = 0

            while not all_done:
                print("About to retrieve data for %s from %s -> %s" % (phone_label, curr_start_ts, r["end_ts"]))
                curr_location_entries = retrieve_data_from_server(phone_label, ["background/location"], curr_start_ts, r["end_ts"])
                print("Retrieved %d entries with timestamps %s..." % (len(curr_location_entries), [cle["data"]["ts"] for cle in curr_location_entries[0:10]]))
                if len(curr_location_entries) == 0 or len(curr_location_entries) == 1 or len(curr_location_entries) == prev_retrieved_count:
                    all_done = True
                else:
                    location_entries.extend(curr_location_entries)
                    curr_start_ts = curr_location_entries[-1]["data"]["ts"]
                    prev_retrieved_count = len(curr_location_entries)
            location_df = pd.DataFrame([e["data"] for e in location_entries])
            location_df["hr"] = (location_df.ts-r["start_ts"])/3600.0
            r["location_df"] = location_df

In [ ]:
count_map = {}
for phoneOS, phone_map in eval_transitions.items():
    print("Processing data for %s phones" % phoneOS)
    for phone_label in phone_map:
        curr_phone_count_map = {}
        curr_calibration_ranges = phone_map[phone_label]["calibration_ranges"]
        for r in curr_calibration_ranges:
            curr_phone_count_map[r["trip_id"]] = len(r["location_df"])
        count_map[phoneOS+"_"+phone_label] = curr_phone_count_map
        
count_df = pd.DataFrame(count_map).transpose()
count_df            

In [ ]:
count_df.plot(kind="bar", figsize=(16,6))

In [ ]:
def get_location_density_df(phone_map):
    density_map = {}
    for phone_label in phone_map:
        curr_phone_density_map = {}
        curr_calibration_ranges = phone_map[phone_label]["calibration_ranges"]
        for r in curr_calibration_ranges:
            density_map[phone_label+"_"+r["trip_id"]] = r["location_df"].hr
        
    density_df = pd.DataFrame(density_map)
    return density_df

In [ ]:
# Density dfs are not adding much at this point
# android_density_df = get_location_density_df(eval_transitions["android"])
# nRows = get_row_count(len(android_density_df), 2)
# print(nRows)
# android_density_df.plot(kind='density', subplots=False, layout=(nRows, 2), figsize=(10,10), sharex=True, sharey=True)

In [ ]:
def plot_separate_density_curves(fig, nRows, phone_map):
    for i, phone_label in enumerate(phone_map.keys()):
        ax = fig.add_subplot(nRows, 2, i+1, title=phone_label)
        curr_calibration_ranges = phone_map[phone_label]["calibration_ranges"]
        for r in curr_calibration_ranges:
            location_df = r["location_df"]
            # print(battery_df.battery_level_pct.tolist())
            location_df.hr.plot(kind='density', ax=ax, label=r["trip_id"], sharex=True, sharey=True)
            ax.legend()

In [ ]:
# And neither are separate curves
# (ifig, ax) = plt.subplots(nrows=0, ncols=0, figsize=(16,16))
# nRows = get_row_count(len(eval_transitions["android"].keys()), 2)
# print(nRows)
# plot_separate_density_curves(ifig, nRows, eval_transitions["android"])

In [ ]:
# ios_density_df = get_location_density_df(eval_transitions["ios"])
# nRows = get_row_count(len(eval_transitions["ios"].keys()), 2)
# print(nRows)
# ios_density_df.plot(kind='density', subplots=False, layout=(nRows, 2), figsize=(10,10), sharex=True, sharey=True)

In [ ]:
# (ifig, ax) = plt.subplots(nrows=0, ncols=0, figsize=(16,16))
# nRows = get_row_count(len(eval_transitions["ios"].keys()), 2)
# print(nRows)
# plot_separate_density_curves(ifig, nRows, eval_transitions["ios"])

In [ ]:
# test_loc_df = eval_transitions["ios"]["ucb-sdb-ios-1"]["calibration_ranges"][0]["location_df"]; test_loc_df.head()

In [ ]:
# test_loc_df = eval_transitions["ios"]["ucb-sdb-ios-2"]["calibration_ranges"][0]["location_df"]; test_loc_df.head()

In [ ]:
def plot_density_vs_power_curves(fig, nRows, phone_map, sel_trip_id):
    for i, phone_label in enumerate(phone_map.keys()):
        ax = fig.add_subplot(nRows, 2, i+1)
        curr_calibration_ranges = phone_map[phone_label]["calibration_ranges"]
        for r in curr_calibration_ranges:
            if r["trip_id"] == sel_trip_id:
                battery_df = r["battery_df"]
                location_df = r["location_df"]
                battery_df.plot(x="hr", y="battery_level_pct", ax=ax, label=phone_label, sharex=True, sharey=True)
                location_df.hr.plot(ax=ax, kind="density", secondary_y=True)

In [ ]:
# fig = plt.figure(figsize=(16, 16))
# plot_density_vs_power_curves(fig, nRows, eval_transitions["android"], CURR_TRIP_ID)

In [ ]:
# fig = plt.figure(figsize=(16, 16))
# plot_density_vs_power_curves(fig, nRows, eval_transitions["ios"], CURR_TRIP_ID)

## Location points over space

In [ ]:
def get_map_list_for_trip():
    map_list = []
    for phoneOS, phone_map in eval_transitions.items():
        print("Processing data for %s phones" % phoneOS)
        for phone_label in phone_map:
            curr_calibration_ranges = phone_map[phone_label]["calibration_ranges"]
            for r in curr_calibration_ranges:
                curr_map = folium.Map()
                location_df = r["location_df"]
                latlng_route_coords = list(zip(location_df.latitude, location_df.longitude))
                # print(latlng_route_coords[0:10])
                pl = folium.PolyLine(latlng_route_coords,
                    popup="%s: %s" % (phoneOS, phone_label))
                pl.add_to(curr_map)
                curr_bounds = pl.get_bounds()
                print(curr_bounds)
                top_lat = curr_bounds[0][0]
                mid_lng = (curr_bounds[0][1] + curr_bounds[1][1])/2
                print("midpoint = %s, %s, plotting at %s, %s" % (top_lat,mid_lng, top_lat, mid_lng))
                folium.map.Marker(
                    [top_lat, mid_lng],
                    icon=fof.DivIcon(
                        icon_size=(200,36),
                        html='<div style="font-size: 12pt; color: green;">%s %s</div>' % (phone_label, r["trip_id"]))
                ).add_to(curr_map)
                curr_map.fit_bounds(pl.get_bounds())
                map_list.append(curr_map)
    return map_list

In [ ]:
ha_map_list = get_map_list_for_trip()

In [ ]:
rows = get_row_count(len(ha_map_list), 2)
evaluation_maps = bre.Figure(ratio="{}%".format((rows/4) * 100))
for i, curr_map in enumerate(ha_map_list):
    evaluation_maps.add_subplot(rows, 2, i+1).add_child(curr_map)
evaluation_maps

## Switch focus to calibration

Target datastructure is

```
- android
    - high_accuracy_train_AO
        - ucb-sdb-android-1
            - battery_df
            - location_df
        - ucb-sdb-android-2
        - ucb-sdb-android-3
        - ucb-sdb-android-4
```

In [ ]:
def find_comparison_regimes():
    comparison_map = {}
    for phoneOS, phone_map in eval_transitions.items(): # android, ios
        print("Processing data for %s phones" % phoneOS)
        comparison_map[phoneOS] = {}
        for phone_label, phone_data_map in phone_map.items(): # ucb-sdb-android-1
            for r in phone_data_map["calibration_ranges"]:
                if r["trip_id"] not in comparison_map[phoneOS]:
                    comparison_map[phoneOS][r["trip_id"]] = {}
                comparison_map[phoneOS][r["trip_id"]][phone_label] = r
    return comparison_map

In [ ]:
comparison_map = find_comparison_regimes()

In [ ]:
print(comparison_map["android"].keys(), comparison_map["ios"].keys())
print(comparison_map["android"]["high_accuracy_train_AO"].keys())

## Per-regime battery and location visualization

In [ ]:
def plot_per_trip_evaluation_curves(fig, nRows, phone_map):
    i = 0
    for curr_calibrate, curr_calibrate_trip_map in phone_map.items(): # high_accuracy_train_AO
        ax = fig.add_subplot(nRows, 2, i+1, title=curr_calibrate)
        i = i+1
        for phone_label, phone_data_map in curr_calibrate_trip_map.items():
            print("Extracting data for %s from map with keys %s" % (phone_label, phone_data_map.keys()))
            battery_df = phone_data_map["battery_df"]
            if len(battery_df) > 0:
                battery_df.plot(x="hr", y="battery_level_pct", ax=ax, label=phone_label, ylim=(0,100), sharey=True)
            else:
                print("no battery data found for %s %s, skipping" % (curr_eval, curr_eval_trip_id))

In [ ]:
(ifig, ax) = plt.subplots(figsize=(16,8),nrows=0,ncols=0)
exp_plot_counts = len(comparison_map["android"])
print(exp_plot_counts)
nRows = get_row_count(exp_plot_counts, 2)
print(nRows)
plot_per_trip_evaluation_curves(ifig, nRows, comparison_map["android"])

In [ ]:
(ifig, ax) = plt.subplots(figsize=(16,8),nrows=0,ncols=0)
exp_plot_counts = len(comparison_map["ios"])
print(exp_plot_counts)
nRows = get_row_count(exp_plot_counts, 2)
print(nRows)
plot_per_trip_evaluation_curves(ifig, nRows, comparison_map["ios"])

In [ ]:
def get_per_trip_map_list():
    map_list = []
    color_list = ['blue', 'red', 'purple', 'orange']
    for phoneOS, phone_map in comparison_map.items():
        print("Processing data for %s phones" % phoneOS)
        for curr_calibrate, curr_calibrate_trip_map in phone_map.items():
            curr_map = folium.Map()
            all_points = []
            for i, (phone_label, phone_data_map) in enumerate(curr_calibrate_trip_map.items()):
                print("%d, %s, %s" % (i, phone_label, phone_data_map.keys()))
                location_df = phone_data_map["location_df"]
                latlng_route_coords = list(zip(location_df.latitude, location_df.longitude))
                all_points.extend(latlng_route_coords)
                # print(latlng_route_coords[0:10])
                if len(latlng_route_coords) > 0:
                    print("Processing %s, %s, found %d locations, adding to map" %
                        (curr_calibrate, phone_label, len(latlng_route_coords)))
                    pl = folium.PolyLine(latlng_route_coords,
                        popup="%s" % (phone_label), color=color_list[i])
                    pl.add_to(curr_map)
                else:
                    print("Processing %s, %s, found %d locations, skipping" %
                        (curr_calibrate, phone_label, len(latlng_route_coords)))
            curr_bounds = ful.get_bounds(all_points)
            print(curr_bounds)
            top_lat = curr_bounds[0][0]
            mid_lng = (curr_bounds[0][1] + curr_bounds[1][1])/2
            print("for trip %s with %d points, midpoint = %s, %s, plotting at %s, %s" %
                    (curr_calibrate, len(all_points), top_lat,mid_lng, top_lat, mid_lng))
            folium.map.Marker(
                [top_lat, mid_lng],
                icon=fof.DivIcon(
                    icon_size=(200,36),
                    html='<div style="font-size: 12pt; color: green;">%s: %s</div>' % (phoneOS, curr_calibrate))
            ).add_to(curr_map)
            curr_map.fit_bounds(pl.get_bounds())
            map_list.append(curr_map)
    return map_list

In [ ]:
ha_map_list = get_per_trip_map_list()

In [ ]:
rows = get_row_count(len(ha_map_list), 2)
print(rows)
evaluation_maps = bre.Figure(ratio="{}%".format((rows/4) * 100))
for i, curr_map in enumerate(ha_map_list):
    # print("Adding map %s at %d" % (curr_map, i))
    evaluation_maps.add_subplot(rows, 2, i+1).add_child(curr_map)
evaluation_maps